In [19]:
import nltk
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
# Read Medium data
top_articles = pd.read_json('top_health_articles.json')
tag = "Health"
sort_color = 'green_positive'
num_articles = 5
top_articles.head()

,date,tags,title,author,claps,length,url,corpus
52,2018-01-17,"[life, design, entrepreneur, ideas, health]",Adversity is the Mother of Invention,Pierre-Emmanuel Boiton,533,7,https://medium.com/@peboiton/adversity-is-the-...,"Four years ago, I had an accident. The kind of..."
104,2018-01-17,"[feminism, sex, rape, health, wellbeing]",5 Stories of Affirmative Consent,Devon Price,1723,8,https://medium.com/@devonprice/5-nice-stories-...,I.He was in town visiting for a conference. We...
131,2018-01-17,"[fitness, wellness, health, exercise, aging]",Injury prevention gets harder as you age. Thes...,Washington Post,567,5,https://medium.com/thewashingtonpost/injury-pr...,"By Carolee Belkin WalkerBaby, it’s cold outsid..."
154,2018-01-17,"[food, psychology, diet, health, motivation]",The Amazing One-Apple-A-Day Diet,Gabriel Iosa,785,3,https://medium.com/@gabrieliosa/the-amazing-on...,"I’m not talking Apple the company, but apple t..."
186,2018-01-16,"[politics, food, history, health, economics]",Top Ramen and the Historical Narratives Impact...,Laura LeMoon,1229,4,https://medium.com/@lauralemoon/top-ramen-and-...,"As I cook my nightly Top Ramen, it occurs to m..."


In [44]:
# Clean corpi for text analysis
def clean(text):
    text = text.lower()
    for char in ['\n', '“', '”']:
        text = text.replace(char, '')
    text = text.rstrip('written bywritten by')
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

top_articles['clean_corpus'] = top_articles['corpus'].apply(clean)
top_articles['clean_corpus'].iloc[0]

'four years ago i had an accident the kind of accident that can change your life this accident has been the starting point for creating a new product i thought that it was the right time to share a little bit more of this story with you todayi was spending the weekend with my friends in aixlesbains aixlesbains is a charming little town in france with a beautiful lake my friends own a boat there and as it was a sunny day we decided to spend the saturday afternoon sailing on the lakeat one point we discovered a nice cliff so we decided to climb it and then jump into the water the water was so crystal clear there it felt so inviting moreover the cliff was not too high around 8 meters so it didn’t look too dangerous my friends jumped first then it was my turn when i jumped i was afraid to hurt my back on the rocks behind me to avoid injuring myself i decided to jump as far as possible but i got unbalanced and landed in the water on my bottomright after the jump my back felt sore i lay down

In [45]:
color_db = {
    'purple_negative': ['mystery', 'mysterious', 'moodiness', 'moody', 'boredom', 'bored', 'bore', 'confusion', 'confuse', 'confused', 'disconnection', 'disconnect'],
    'purple_positive': ['connection', 'connect', 'wisdom', 'wise', 'spirituality', 'spiritual', 'royalty', 'royal, ''nobility', 'noble', 'luxury', 'luxurious', 'ambition', 'ambitious', 'wealth', 'wealthy', 'awaken', 'awake'],
    'blue_negative': ['coldness', 'cold', 'masculinity', 'masculine', 'male', 'disgust', 'disgusted', 'conflict', 'conflicting', 'aggression', 'aggressive'],
    'blue_positive': ['intuition', 'imagination', 'imagine', 'tranquility', 'tranquil', 'security', 'secure', 'integrity', 'peace', 'peaceful', 'loyalty', 'loyal', 'faith', 'faithful', 'intelligence', 'intelligent'],
    'teal_negative': ['femininity', 'feminine', 'female', 'hostility', 'hostile'],
    'teal_positive': ['communication', 'communicate', 'expression', 'express', 'healing', 'heal', 'protection', 'protect', 'sophisticated', 'cleanse', 'cleansing'],
    'green_negative': ['envy', 'envious', 'jealousy', 'jealous', 'guilt', 'guilty', 'fear', 'fearful', 'scared', 'judgmental', 'judge', 'judging', 'unforgiving', 'anxiety', 'anxious'],
    'green_positive': ['compassion', 'trust', 'freshness', 'fresh', 'environment', 'new', 'money', 'fertile', 'health', 'healthy', 'grounded', 'reconnecting', 'balanced', 'balance', 'balancing'],
    'yellow_negative': ['irresponsible', 'instability', 'grief', 'grieve', 'grieving', 'grieves', 'addiction', 'addict', 'addicted', 'insecurity', 'insecure', 'depression', 'depressed'],
    'yellow_positive': ['confident', 'confidence', 'bright', 'sunny', 'energetic', 'warm', 'happy', 'happiness', 'perky', 'joy', 'joyful', 'intellect', 'intellectual'],
    'orange_negative': ['ignorance', 'ignorant', 'sluggishness', 'sluggish', 'shame', 'ashamed', 'shameful', 'compulsiveness', 'compulsive', 'loneliness', 'lonely', 'alone', 'dependence', 'dependent'],
    'orange_positive': ['courage', 'friendliness', 'friendly', 'success', 'successful', 'creativity', 'creative', 'openness', 'open', 'sexual', 'sexy', 'sex'],
    'red_negative': ['anger', 'angry', 'unsafe', 'warned', 'warn', 'warning', 'worry', 'worried', 'volatile', 'hopelessness', 'hopeless'],
    'red_positive': ['love', 'loving', 'passion', 'passionate', 'energy', 'energetic', 'power', 'powerful', 'strength', 'strong', 'heat', 'hot', 'desire', 'safe', 'safety', 'instinctive', 'instinct', 'security', 'secure', 'liberating', 'liberate']
}

def count_color(text):
    words = text.split()
    color_words = [word for word in words if word in color_db[sort_color]]
    return(len(color_words))
           
#count_color(top_articles['clean_corpus'].iloc[0])
top_articles['color_count'] = top_articles['clean_corpus'].apply(count_color)
sort_color_articles = top_articles.sort_values('color_count', ascending=False)
sort_color_articles

,date,tags,title,author,claps,length,url,corpus,clean_corpus,color_count
65444,2019-04-15,"[politics, finance, life-lessons, news, health]",In Reality: A New Worldview Just for You,David Siegel,1120,81,https://medium.com/@pullnews/in-reality-a-new-...,"THANK YOU FOR THE MORE THAN 1,000 CLAPS! I’M G...",thank you for the more than 1000 claps i’m gla...,116
109701,2019-12-31,"[climate-change, system-thinking, coexistence,...",THE RISE OF THE ECO INDIVIDUAL,Rasmus Rask,913,20,https://medium.com/@rasmusrask.eco/the-rise-of...,The health of the individual is inseparable fr...,the health of the individual is inseparable fr...,58
40303,2018-10-12,"[obesity, wellness, health, exercise, nutrition]",Breaking New Science! 10 Strange Tricks to Los...,"Daniel J. Schultz, MS, RD",1334,13,https://medium.com/@VegetableManDan/breaking-n...,"And now, I’ve got your attention. Apologies fo...",and now i’ve got your attention apologies for ...,42
107876,2020-01-13,"[activism, equality, bodypostivity, health, ve...","Health Is a Privilege, Not a Virtue",Emma Green,782,6,https://medium.com/tenderlymag/healthism-in-ve...,"Healthism, a term coined in 1980 by political ...",healthism a term coined in 1980 by political e...,37
73877,2019-05-30,"[menopause, alzheimers, health, womens-health,...",Exploring the Link Between Menopause and Alzhe...,Deborah Copaken,16460,37,https://medium.com/neurotrack/menopause-and-al...,"It started with an artichoke. Or rather, it st...",it started with an artichoke or rather it star...,35
...,...,...,...,...,...,...,...,...,...,...
100315,2019-11-24,"[multiple-sclerosis, mental-health, habit-buil...",My Most Foundational Habit,"Adam, Diabetic Cyborg",651,1,https://medium.com/muddyum/my-most-foundationa...,In response to Susan Brearley’s promptMy most ...,in response to susan brearley’s promptmy most ...,0
120288,2020-03-21,"[corona, coronavirus, politics, health]",It's rare to see someone put this much effort ...,Adam T,626,1,https://medium.com/@adam.thimons/its-rare-to-s...,It's rare to see someone put this much effort ...,its rare to see someone put this much effort i...,0
55249,2019-02-05,"[lose-weight, running-tips, health]",Туранхай байх уу? VS эрүүл байх уу?,Hotolsar,1170,6,https://medium.com/@hotolsar/%D1%82%D1%83%D1%8...,Балетын сургуульд охидыг шалгаруулж авахдаа ээ...,балетын сургуульд охидыг шалгаруулж авахдаа ээ...,0
26191,2018-06-18,"[life, life-lessons, inspiration, health, rela...",Airplane Mode Strengthened My Marriage,Danny Forest,549,4,https://medium.com/@danny_forest/airplane-mode...,"A few weeks ago, my wife was feeling down and ...",a few weeks ago my wife was feeling down and d...,0


In [46]:
list(sort_color_articles.iloc[0:num_articles]['url'])

['https://medium.com/@pullnews/in-reality-a-new-worldview-just-for-you-ea72eeea04b1',
 'https://medium.com/@rasmusrask.eco/the-rise-of-the-eco-individual-9b920a2c31bb',
 'https://medium.com/@VegetableManDan/breaking-new-science-10-strange-tricks-to-lose-belly-fat-incredibly-fast-73580bb65c0b',
 'https://medium.com/tenderlymag/healthism-in-veganism-its-time-to-call-it-out-174903d477da',
 'https://medium.com/neurotrack/menopause-and-alzheimers-1c455f29fe16']